In [1]:
import os
import re
import requests
import statistics
from src.najdi_rok import najdi_rok
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [3]:
kam = 'data_raw/nobelistvo'

In [4]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [5]:
koho = pd.read_json(os.path.join('data_raw','wikidata_nobelovky_pro_filtr.json'))

In [6]:
koho.sample(5)

,024_a,nobelovka,100_7
34,Q82248,1932,jn19990002542
114,Q317877,2021,xx0011793
110,Q272855,2017,jn19990003916
9,Q44519,1909,jn19990004817
54,Q80064,1981,jn19990001316


In [7]:
sloupce = ['100','245','700']

In [8]:
df = pd.read_parquet(os.path.join('data/cnb_sloupce','008.parquet'))
for s in sloupce:
    print(s)
    df = df.merge(pd.read_parquet(os.path.join('data/cnb_sloupce',f'{s}.parquet')), left_index=True, right_index=True, how='outer')
    print(len(df))

100
1238228
245
1238228
700
1238228


In [9]:
df.sample(10)

,008,100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,245_v,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_n,700_k,700_m,700_r,700_p,700_o,700_s,700_e,700_6,700_x,700_f,700_g
001,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bknvla01255,[040726s1895 xr e 000 0 cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Za pravdu boží a jazyk slovanský /],None,[Upr. Jan Hejret],None,None,None,None,None,None,[1],"[Hejret, Jan,]",[edt],[1868-1947],[jk01040593],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
nkc20081835895,[081216s2009 xr d p 000 0 cze],[1],"[Olejnik, Donata]",[xx0089989],[aut],None,None,None,None,None,[1],[0],[Angličtina 3 v 1 :],"[maturitní minimum : gramatika, témata, konver...",[[text Donata Olejnik]],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk20031250451,[030521s1900 xr e 000 d cze],[1],"[Kádner, Karel,]",[jk01052486],[lbt],[1859-1923],None,None,None,None,[1],[0],[Stáňa :],[libreto k opeře o jednom dějství /],[dle Štolbovy humoresky napsal Karel Kádner ; ...,None,None,None,None,None,None,"[1, 1]","[Štolba, Josef,, Malát, Jan,]","[ant, cmp]","[1846-1930, 1843-1915]","[jk01131452, jk01073183]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
cpk19950113843,[960502s1995 xr e 000 jbcze],[1],"[Klempera, Josef,]",[jk01060396],[aut],[1926-2011],None,None,None,None,[1],[0],[Smrt v pralese /],None,[Josef Klempera],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk194701181,[980720s1947 xr ||| | cze],[1],"[Szmaglewska, Seweryna,]",[ola2003198060],[aut],[1916-1992],None,None,None,None,[1],[0],[Dýmy nad Birkenau /],None,[Seweryna Szmaglewska ; Z polštiny [Dymy nad B...,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk195302512,[980317s1953 xr rus],[1],"[Žebera, Karel,]",[jk01153073],[aut],[1911-1986],None,None,None,None,[1],[0],[Drevnejšije pamjatniki raboty čeloveka v Čech...,None,[Karel Žebera ; Z češtiny Nejstarší památky li...,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bknvsv06894,[041203s1938 xr z a 0 cze],[1],"[Semík, Matěj,]",[jk01110752],[ctg],[1869-1961],None,None,None,None,[1],[0],[Česko-slovenská republika /],None,[zpracoval M. Semík],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20112255764,[111122s2011 xr g 000 j cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Kouzlo zimy],None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk20021197565,[021217s1856 xr e 000 m cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Písně ke mssi swaté uspořádané dle církewniho...,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for index, row in koho.iterrows():
    print(row['nobelovka'])
    df_slice = df[df['100_7'].astype(str).str.contains(row['100_7'], na=False) | df['700_7'].astype(str).str.contains(row['100_7'], na=False)]
    df_slice = df_slice.explode('008')
    df_slice['rok'] = df_slice['008'].apply(lambda x: najdi_rok(x))
    df_slice['nobelovka'] = row['nobelovka']
    df_slice['titul'] = df_slice['245_a'].apply(lambda x: x[0].replace(' /','').replace(' :','').replace(';','').strip() if x != None else None)
    df_slice.to_parquet(os.path.join(kam, f"nobelovka_{row['nobelovka']}_{row['100_7']}.parquet"))

1901
1902
1903
1904
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1915
1916
1917
1917
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1944
1961
1999
